In [1]:
from glob import glob
from helpers import create_polling_id
import json
import os
import pandas as pd
import re

os.chdir("../")

In [15]:
election_type = "camera"
year = 2001
# https://elezionistorico.interno.gov.it/eligendohome/opendata.php

In [14]:
pd.read_csv(f"data/Italy/{election_type}-{2001}.zip", header=0, delimiter=";")

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['camera-20010513_Proporzionale.txt', 'Camera_20010513_Uninom_Cand&Contr.txt', 'Camera_20010513_Uninom_Scrutini.txt']

In [16]:
import zipfile
import pandas as pd
with zipfile.ZipFile(f"data/Italy/{election_type}-{year}.zip") as z:
    with z.open("Camera_Italia_LivComune.txt") as f:
        df = pd.read_csv(f, header=0, delimiter=";")

df["candidate"] = df["NOME"] + " " + df["COGNOME"]
df["polling_id"] = create_polling_id(df, columns=["CODTIPOELEZIONE", "CIRC-REG", "COLLPLURI", "COLLUNINOM", "COMUNE"])
df.head()

KeyError: "There is no item named 'Camera_Italia_LivComune.txt' in the archive"

In [16]:
df.groupby(["candidate", "DESCRLISTA"]).agg({"VOTICANDIDATO": "sum", "VOTILISTA": "sum"}).sort_values("VOTICANDIDATO")

VOTICANDIDATO  \
candidate         DESCRLISTA                                                 
ALESSANDRA FILICE FORZA DEL POPOLO                                      61   
LAURA CARERI      FORZA DEL POPOLO                                      78   
PATRIZIO MARALDI  MASTELLA NOI DI CENTRO EUROPEISTI                    106   
CARLO FIUMANO'    FORZA DEL POPOLO                                     109   
RITA  ROCCO       MASTELLA NOI DI CENTRO EUROPEISTI                    117   
...                                                                    ...   
CIRO MASCHIO      LEGA PER SALVINI PREMIER                          144942   
ALESSANDRO SORTE  FRATELLI D'ITALIA CON GIORGIA MELONI              153202   
                  FORZA ITALIA                                      153202   
                  NOI MODERATI/LUPI - TOTI - BRUGNARO - UDC         153202   
                  LEGA PER SALVINI PREMIER                          153202   

                                                             VOTILISTA  
candidate         DESCRLISTA                                            
ALESSANDRA FILICE FORZA DEL POPOLO                                  60  
LAURA CARERI      FORZA DEL POPOLO                                  74  
PATRIZIO MARALDI  MASTELLA NOI DI CENTRO EUROPEISTI                 96  
CARLO FIUMANO'    FORZA DEL POPOLO                                 103  
RITA  ROCCO       MASTELLA NOI DI CENTRO EUROPEISTI                103  
...                                                                ...  
CIRO MASCHIO      LEGA PER SALVINI PREMIER                       34467  
ALESSANDRO SORTE  FRATELLI D'ITALIA CON GIORGIA MELONI           79978  
                  FORZA ITALIA                                   21546  
                  NOI MODERATI/LUPI - TOTI - BRUGNARO - UDC       1620  
                  LEGA PER SALVINI PREMIER                       46219  

[2179 rows x 2 columns]

In [9]:
df_filtered = df.rename(columns={"VOTILISTA": "value"})

tt = df_filtered.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate"]]

# df_filtered = pd.merge(df_filtered, tt, on=["polling_id", "candidate"])
# df_filtered.head()

In [10]:
tt.head()

,polling_id,candidate,rate
0,C-ABRUZZO-ABRUZZO - P01-ABRUZZO - U01 (CHIETI)...,ALBERTO BAGNAI,0.550698
1,C-ABRUZZO-ABRUZZO - P01-ABRUZZO - U01 (CHIETI)...,AMANDA DE MENNA,0.013023
2,C-ABRUZZO-ABRUZZO - P01-ABRUZZO - U01 (CHIETI)...,CARMELA GRIPPA,0.139535
3,C-ABRUZZO-ABRUZZO - P01-ABRUZZO - U01 (CHIETI)...,DONATO TROVARELLI,0.001860
4,C-ABRUZZO-ABRUZZO - P01-ABRUZZO - U01 (CHIETI)...,EDDA GIUBERTI,0.000930
